# IBM Instance Setup

In [1]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.backend("ibm_aachen")
noise_model = NoiseModel.from_backend(backend)

# Experiments Poster

In [3]:
NODES = 10
SP = "Sparse"
MD = "Medium"
DS = "Dense"

ER = "ER"
BP = "Bipartite"

In [4]:
ER8NSParams = np.load("results/params/params_8N_ER_Sparse_LR0.04522709518095061_SIGMA0.6587519702447917_INIT0.17944082088572766_NUMLAYERS1.npy")
ER8NMParams = np.load("results/params/params_8N_ER_Medium_LR0.011220685773844122_SIGMA0.6962179567887462_INIT1.810103830663358_NUMLAYERS5.npy")
ER8NDParams = np.load("results/params/params_8N_ER_Dense_LR0.04206675941389984_SIGMA0.7206244967648459_INIT0.5521825859396434_NUMLAYERS1.npy")
BP8NSParams = np.load("results/params/params_8N_Bipartite_Sparse_LR0.042174583320667126_SIGMA0.8175072163174264_INIT1.0464881642904231_NUMLAYERS1.npy")
BP8NMParams = np.load("results/params/params_8N_Bipartite_Medium_LR0.003551614405932626_SIGMA1.3352557243570238_INIT0.8853914111571922_NUMLAYERS5.npy")
BP8NDParams = np.load("results/params/params_8N_Bipartite_Dense_LR0.0018973825577720538_SIGMA1.3106607895258173_INIT0.1942402847384792_NUMLAYERS3.npy")

## Experiments

In [5]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()
dev = qml.device("lightning.qubit", 
                    wires=QUBITS, 
                    shots=512)

gate_structure = efficient_connectivity_gates(grid_conn, QUBITS, 1)

In [6]:
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_aer.noise import NoiseModel
from qiskit.visualization import plot_histogram
from qiskit_ibm_provider import IBMProvider

import numpy as np
from qiskit.circuit import QuantumCircuit, Parameter

def create_iqp_circuit(n_qubits: int, gates: list[list[list[int]]]) -> tuple[QuantumCircuit, list[Parameter]]:
    """
    Constructs a Qiskit circuit with an IQP-like structure.

    Args:
        n_qubits (int): The number of qubits in the circuit.
        gates (list[list[list[int]]]): Specification of the trainable gates.
            - Each element of `gates` corresponds to a unique trainable parameter.
            - Each sublist specifies the generators (Pauli strings) for that parameter.
            - A generator with one qubit [q] is an RZ gate.
            - A generator with two qubits [q1, q2] is an RZZ gate.

    Returns:
        A tuple containing the quantum circuit with symbolic parameters
        and the list of those parameter objects.
    """
    # 1. Initialize the quantum circuit and create symbolic parameters
    qc = QuantumCircuit(n_qubits)
    params = [Parameter(f'θ_{i}') for i in range(len(gates))]

    # 2. Add the first Hadamard layer
    qc.h(range(n_qubits))
    qc.barrier()

    # 3. Add the parameterized RZ and RZZ gates
    for i, gate_group in enumerate(gates):
        for generator in gate_group:
            # The MultiRZ(phi, wires) in PennyLane corresponds to RZ or RZZ gates
            # with a rotation of `phi`. In Qiskit, the angle is specified directly.
            # Your original code used `2*par`, so we use `2*params[i]` here for consistency.
            angle = 2 * params[i]
            if len(generator) == 1:
                qc.rz(angle, generator[0])
            elif len(generator) == 2:
                qc.rzz(angle, generator[0], generator[1])
    qc.barrier()

    # 4. Add the final Hadamard layer
    qc.h(range(n_qubits))
    qc.barrier()

    # 5. Add measurement in the computational basis
    qc.measure_all()

    return qc, params

/var/folders/6q/mzgmvrhn3l76p312t6zn3mxw0000gn/T/ipykernel_49063/662024288.py:4: DeprecationWarning: The package qiskit_ibm_provider is being deprecated. Please see https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime to get instructions on how to migrate to qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from qiskit_ibm_provider import IBMProvider


In [7]:
trained_params = np.array(BP8NSParams)
iqp_circuit, circuit_params = create_iqp_circuit(QUBITS, gate_structure)
bound_circuit = iqp_circuit.assign_parameters(
    {param: value for param, value in zip(circuit_params, trained_params)}
)

In [8]:
noisy_sampler = AerSampler(
    backend_options={
        'noise_model': noise_model,
        'method': 'matrix_product_state'
    }
)

In [9]:
print("▶️ Running noisy simulation...")
job = noisy_sampler.run(bound_circuit, shots=512)
result = job.result()

▶️ Running noisy simulation...


In [10]:
# 1. Get the quasi-distribution for the first (and only) circuit run
quasi_distribution = result.quasi_dists[0]

# 2. Get the dictionary of outcomes {bitstring: probability}
counts_dict = quasi_distribution.binary_probabilities()

# 3. Extract the bitstrings by taking the keys of the dictionary 🔑
resulting_bitstrings = list(counts_dict.keys())


# --- Print the results ---
print("\nFull dictionary of outcomes:")
print(counts_dict)

print("\nList of resulting bitstrings:")
print(resulting_bitstrings)


Full dictionary of outcomes:
{'0000000100011100101000000100': 0.001953125, '0000001000010000111000000011': 0.001953125, '0000000000000000110101001001': 0.001953125, '0000000000100000101101100010': 0.001953125, '0000000000100100000010111000': 0.001953125, '0000000000000100010000100001': 0.001953125, '0000000010111011001100101111': 0.001953125, '0000000000010001111000101100': 0.001953125, '1000000000001010001001000010': 0.001953125, '0000000000010000100001101111': 0.001953125, '0000000000011101000001010100': 0.001953125, '0000100000000000010100010110': 0.001953125, '0000000000011000100100101011': 0.001953125, '0000000000111011000100001100': 0.001953125, '0000000001000000100011000010': 0.001953125, '0000000100100011001000100001': 0.001953125, '0000000000100100001010100100': 0.001953125, '0000000000010000100000100110': 0.001953125, '0000001100110101001010011000': 0.001953125, '0000000000011000001100000001': 0.001953125, '0000100100100100101110010000': 0.001953125, '00000000011100010010011

In [11]:
uint8_arrays = [np.array([int(bit) for bit in b_str], dtype=np.uint8) for b_str in resulting_bitstrings]
np.save("results/samples/qiskit/BP8NS_simulated.npy", uint8_arrays)

In [12]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=150)

In [13]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(mode=backend)

In [14]:
sampler.options.resilience_level = 1

ValidationError: 1 validation error for SamplerOptions
resilience_level
  Object has no attribute 'resilience_level' [type=no_such_attribute, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/no_such_attribute

In [ ]:
from qiskit.compiler import transpile

# 1. Initialize the Sampler and set options
sampler = Sampler(mode=backend)
sampler.options.experimental = {"m3": True} # For readout error mitigation

# 2. Remove final measurements from the original circuit
circuit_to_run = bound_circuit.remove_final_measurements(inplace=False)

# 3. TRANSPILE the circuit for the backend (THIS IS THE FIX)
# Use optimization_level=3 for the best performance on hardware.
isa_circuit = transpile(circuit_to_run, backend=backend, optimization_level=3)

# 4. Run the transpiled circuit
job = sampler.run([isa_circuit], shots=10000)

print(f"Job submitted with ID: {job.job_id()}")

# 5. Get and plot the results
result = job.result()
mitigated_counts = result[0].data.meas.get_counts()

/Users/uribagi/Documents/GitHub/Latent-IQP/.venv/lib/python3.12/site-packages/qiskit_ibm_runtime/utils/validations.py:40: UserWarning: The 0-th circuit has no output classical registers so the result will be empty. Did you mean to add measurement instructions?
  warnings.warn(


Job submitted with ID: d1hnmkot4q0s739pgfeg


RuntimeJobFailureError: "Unable to retrieve job result. Error code 1518; Error(s) when trying to set an option in SamplerOptions:\\n * m3: Object has no attribute 'm3'. -- Ensure the options are specified correctly and retry. To learn more, see the `Specify options <https://quantum.cloud.ibm.com/docs/guides/specify-runtime-options#specify-options>`__ documentation. -- https://ibm.biz/error_codes#1518"

In [ ]:
with Session(service=service, backend=backend) as session:
    # 2. Pass the options dictionary directly into the Sampler constructor

    
    # Execute the job
    print("\n▶️ Running your circuit on hardware with error mitigation...")
    job = sampler.run(bound_circuit, shots=512)
    print(f"✅ Job submitted with ID: {job.job_id()}")

    # Retrieve and process the results
    result = job.result()
    mitigated_counts = result.quasi_dists[0].binary_probabilities()
    print("✅ Job complete.")

/var/folders/6q/mzgmvrhn3l76p312t6zn3mxw0000gn/T/ipykernel_17126/288642469.py:1: DeprecationWarning: The service parameter is deprecated as of qiskit-ibm-runtime 0.26.0 and will be removed no sooner than 3 months after the release date. The service can be extracted from the backend object so it is no longer necessary.
  with Session(service=service, backend=backend) as session:
/var/folders/6q/mzgmvrhn3l76p312t6zn3mxw0000gn/T/ipykernel_17126/288642469.py:3: DeprecationWarning: The 'session' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(session=session, options={"resilience_level": 2})


ValidationError: 1 validation error for SamplerOptions
resilience_level
  Unexpected keyword argument [type=unexpected_keyword_argument, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument